In [1]:
from copy import deepcopy
from typing import Tuple, List
import math
import itertools
from IPython.display import HTML, display
from ipywidgets import HTML as WidgetHTML
import random
import tkinter as tk
from tkinter import messagebox

In [2]:
class Tablerowumpus:
    
    def __init__(self, matrix = None, flechas = 1):
        self.setMatrix(matrix)
        self.flechas = flechas
    
    
    def __eq__(self, other) -> bool:


        if len(self.matrix) != len(other.getMatrix()):
            return False
        for fila1, fila2 in zip(self.matrix, other.getMatrix()):
            if len(fila1) != len(fila2):
                return False
                
            for casilla1, casilla2 in zip(fila1, fila2):
                if casilla1 != casilla2:
                    return False

        return True
                
    
    def setMatrix(self, matrix):
        self.matrix = deepcopy(matrix)
    
    
    def getMatrix(self) -> List[List]:
        return deepcopy(self.matrix)

    def getFlechas(self):
        return self.flechas
    
    def placeTile(self, row: int, col: int, tile: int):
        self.matrix[row-1][col-1] = tile
    
    
    def utility(self) -> float:

        valor = 0 #Valor inicial del tablero
        if self.flechas < 1:#Para asegurarnos de que no gasta la flecha sin objetivo
            valor -= 10 #Si gasta una flecha resta puntos
        
        for i, fila in enumerate(self.matrix):
            for j, casilla in enumerate(fila):
                if casilla == 1: #El agente en una casilla vacia, guardamos su posicion, ni resta ni suma
                    posAgente = (i, j)
                elif casilla == 11:#Si el agente esta en hedor, resta puntos
                    valor -= 2
                    posAgente = (i, j)
                elif casilla == 12: #Si el agente esta en brisa, resta puntos
                    valor -= 2
                    posAgente = (i, j)
                elif casilla == 13: #Si el agente esta en brisa y hedor, resta puntos
                    valor -= 4
                    posAgente = (i, j)
                elif casilla == 14: #Si el agente ha caido en un hueco, resta muchos puntos, no queremos que caiga a huecos
                    valor -= 100000
                    posAgente = (i, j)
                elif casilla == 15: #Si el agente esta en la casilla del wumpus, resta muchos puntos
                    valor -= 100000
                    posAgente = (i, j)
                elif casilla in [16, 17, 18, 19]: #Si el agente llega al oro, suma muchos puntos, queremos que llegue al oro
                    valor += 1000
                    posAgente = (i, j)
                    posOro = posAgente

                if casilla in [4, 8, 9, 10]:
                    posOro = (i, j)

                if casilla == 2: #Si el wumpus sigue vivo resta puntos, en caso de que el agente mate al wumpus esto no restara, por ello beneficia que mate al wumpus
                    valor -= 30
                    
        if posAgente != posOro:#cuanto menos sea la distancia, la division sumara un mayor valor, es contrariamente proporcional a la distancia el valor que se suma           
            valor += 50/(abs(posAgente[0]-posOro[0]) + abs(posAgente[1]-posOro[1])) #Sumo 10/la distancia entre el agente y el oro (cuanto mas cerca este mas suma)

        return valor
    
    

    def posiciones_adyacentes(self, x, y, dimension):
        celdas_ady = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        celdas_validas = []
        
        for cx, cy in celdas_ady:
            ni, nj = x + cx, y + cy
            # Verificar si la celda adyacente (ni, nj) está dentro de los límites
            if 0 <= ni < dimension and 0 <= nj < dimension:
                celdas_validas.append((ni, nj))
        return celdas_validas

    def generarTableroInicial(self, dimension):
        
        # Crear el tablero vacío
        tablero = [[0 for _ in range(dimension)] for _ in range(dimension)] 

        # Posición del "agente"
        pos_agente = (dimension-1, 0)
        tablero[dimension-1][0] = 1

        
        #asignar posicion para el "oro"
        while True:
            x, y = random.randint(0, dimension-1), random.randint(0, dimension-1)
            #Comprobamos que este vacia la celda
            if tablero[x][y] == 0 and (x, y) not in self.posiciones_adyacentes(*pos_agente, dimension):
                tablero[x][y] = 4
                break
        
        # asignar posición para el "wumpus"
        while True:
            x, y = random.randint(0, dimension-1), random.randint(0, dimension-1)
            #Comprobamos que este vacia la celda y no este al lado del agente
            if tablero[x][y] == 0 and (x, y) not in self.posiciones_adyacentes(*pos_agente, dimension):#* sirve para desempacar los elementos en pos agente
                tablero[x][y] = 2
                pos_wumpus = (x, y)
                break
                
        #Asignamos hedor a las casillas adyacentes
        for casilla in self.posiciones_adyacentes(*pos_wumpus, dimension):
            if tablero[casilla[0]][casilla[1]] == 4:
                tablero[casilla[0]][casilla[1]] = 8 #hedor + oro
            else:
                tablero[casilla[0]][casilla[1]] = 3 #hedor
                
        # asignar posición para los "huecos"
        numero_huecos = 0
        while numero_huecos < 2:
            x, y = random.randint(0, dimension-1), random.randint(0, dimension-1)
            #Comprobamos que este vacia la celda y no este al lado del wumpus
            if tablero[x][y] == 0 and (x, y) not in self.posiciones_adyacentes(*pos_wumpus, dimension) and (x, y) not in self.posiciones_adyacentes(*pos_agente, dimension):
                tablero[x][y] = 6
                numero_huecos += 1
                for casilla in self.posiciones_adyacentes(x, y, dimension):
                    if tablero[casilla[0]][casilla[1]] == 0:
                        tablero[casilla[0]][casilla[1]] = 5 #brisa
                    elif tablero[casilla[0]][casilla[1]] == 1:
                        tablero[casilla[0]][casilla[1]] = 12#Agente + brisa
                    elif tablero[casilla[0]][casilla[1]] == 3:
                        tablero[casilla[0]][casilla[1]] = 7#Brisa + hedor
                    elif tablero[casilla[0]][casilla[1]] == 4:
                        tablero[casilla[0]][casilla[1]] = 10#brisa + oro
                    elif tablero[casilla[0]][casilla[1]] == 8:
                        tablero[casilla[0]][casilla[1]] = 9#hedor + brisa + oro
                    #No se puede dar la situacion de agente + hedor pq no puede estar el wumpus al lado del agente al comenzar la partida
                    #En las situaciones en las que tenga que haber brisa en una casilla que ya haya brisa o las que haya hueco se queda igual

        self.setMatrix(tablero) 


    def movimientoHuecosDisponibles(self):
        movimientosDisponibles = []
        casillasHuecos = []
        #posWumpus = None
        for i, fila in enumerate(self.matrix):
            for j, casilla in enumerate(fila):
                if casilla in [0, 1, 5, 6, 12, 14]: #evitamos casillas con wumpus, oro, hedor
                    movimientosDisponibles.append((i, j))
                if casilla in [6, 14]:
                    casillasHuecos.append((i, j))
    
        return movimientosDisponibles, casillasHuecos

    def gameOver(self):
        for fila in self.matrix:
            for casilla in fila:
                if casilla in [15, 16, 17, 18, 19]:
                    return True

    def moverHuecos(self):

        movimientosDisponibles, casillasHuecos = self.movimientoHuecosDisponibles()
        
        movimientosGanadores = []

        movimientosGanadores = random.sample(movimientosDisponibles, 2) #elige 2 movimientos aleatorios de movimientosDisponibles

        #Quitamos los huecos viejos y quitamos sus brisas
        for casillaHueco in casillasHuecos:
            if casillaHueco in movimientosGanadores:
                movimientosGanadores.remove(casillaHueco)#evitamos hacer nada si el hueco no se mueve (no eliminamos ni añadimos brisas)
                if not movimientosGanadores:
                    return #si los huecos no se mueven terminamos la funcion
            else:
                if self.matrix[casillaHueco[0]][casillaHueco[1]] == 14:#si el agente esta en el hueco
                    self.matrix[casillaHueco[0]][casillaHueco[1]] = 1 #se queda solo el agente
                else:
                    self.matrix[casillaHueco[0]][casillaHueco[1]] = 0 #desaparece el hueco

                self.eliminarBrisas(casillaHueco)

        #Añadimos huecos nuevos y añadimos sus brisas
        for casilla in movimientosGanadores:
            x, y = casilla
            if self.matrix[x][y] == 0:#no hay nada
                self.matrix[x][y] = 6
            elif self.matrix[x][y] == 1:#agente
                self.matrix[x][y] = 14
            elif self.matrix[x][y] == 5:#brisa
                self.matrix[x][y] = 6
            elif self.matrix[x][y] == 12:#agente y brisa
                self.matrix[x][y] = 14

            self.añadirBrisas(casilla)
                
    def eliminarBrisas(self, casillaHueco):

        posicionesBrisas = self.posiciones_adyacentes(*casillaHueco, len(self.matrix))

        for brisa in posicionesBrisas:
            x, y = brisa
            if self.matrix[x][y] == 5:#brisa sola
                self.matrix[x][y] = 0
            elif self.matrix[x][y] == 7:#brisa + hedor
                self.matrix[x][y] = 3
            elif self.matrix[x][y] == 9:#brisa, hedor y oro
                self.matrix[x][y] = 8
            elif self.matrix[x][y] == 10: #brisa y oro
                self.matrix[x][y] = 4
            elif self.matrix[x][y] == 12:#brisa y agente
                self.matrix[x][y] = 1
            elif self.matrix[x][y] == 13:#brisa, hedor y agente
                self.matrix[x][y] = 11
            elif self.matrix[x][y] == 18:#brisa, hedor, oro y agente
                self.matrix[x][y] = 17
            elif self.matrix[x][y] == 19:#brisa, agente y oro
                self.matrix[x][y] = 16

    def añadirBrisas(self, casillaHueco):

        casillas = self.posiciones_adyacentes(*casillaHueco, len(self.matrix))
        
        for casilla in casillas:
            x, y = casilla
            if self.matrix[x][y] == 0:#brisa sola
                self.matrix[x][y] = 5
            elif self.matrix[x][y] == 3:#brisa + hedor
                self.matrix[x][y] = 7
            elif self.matrix[x][y] == 8:#brisa, hedor y oro
                self.matrix[x][y] = 9
            elif self.matrix[x][y] == 4: #brisa y oro
                self.matrix[x][y] = 10
            elif self.matrix[x][y] == 1:#brisa y agente
                self.matrix[x][y] = 12
            elif self.matrix[x][y] == 11:#brisa, hedor y agente
                self.matrix[x][y] = 13
            elif self.matrix[x][y] == 17:#brisa, hedor, oro y agente
                self.matrix[x][y] = 18
            elif self.matrix[x][y] == 16:#brisa, agente y oro
                self.matrix[x][y] = 19
                
    def movimientosAgenteDisponibles(self):

        for i, fila in enumerate(self.matrix):
            for j, casilla in enumerate(fila):
                if casilla in [1, 11, 12, 13, 14, 15, 16, 17, 18, 19]:
                    x, y = i, j
                    
                    
        movimientosDisponibles = self.posiciones_adyacentes(x, y, len(self.matrix))

        return movimientosDisponibles

    

    def moverAgente(self, casillaDestino):

        for i, fila in enumerate(self.matrix):
            for j, casilla in enumerate(fila):
                if casilla in [1, 11, 12, 13, 14, 15, 16, 17, 18, 19]:
                    x1, y1 = i, j #Pos agente
                    
        x2, y2 = casillaDestino

        #Primero borramos al agente de su posicion actual
        if self.matrix[x1][y1] == 1:#Solo agente
            self.matrix[x1][y1] = 0
        elif self.matrix[x1][y1] == 11:#Agente y hedor
            self.matrix[x1][y1] = 3
        elif self.matrix[x1][y1] == 12:#Agente y brisa
            self.matrix[x1][y1] = 5
        elif self.matrix[x1][y1] == 13:#Agente, hedor y brisa
            self.matrix[x1][y1] = 7
        elif self.matrix[x1][y1] == 14:#Agente y hueco
            self.matrix[x1][y1] = 6
        elif self.matrix[x1][y1] == 15:#Agente y wumpus
            self.matrix[x1][y1] = 2
        elif self.matrix[x1][y1] == 16:#Agente y oro
            self.matrix[x1][y1] = 4
        elif self.matrix[x1][y1] == 17:#Agente hedor y oro
            self.matrix[x1][y1] = 8
        elif self.matrix[x1][y1] == 18:#Agente hedor brisa y oro
            self.matrix[x1][y1] = 9
        elif self.matrix[x1][y1] == 19:#Agente brisa y oro
            self.matrix[x1][y1] = 10

        #Ahora añadimos el agente en la nueva posicion
        if self.matrix[x2][y2] == 0:#Nada
            self.matrix[x2][y2] = 1
        elif self.matrix[x2][y2] == 3:#hedor
            self.matrix[x2][y2] = 11
        elif self.matrix[x2][y2] == 5:#brisa
            self.matrix[x2][y2] = 12
        elif self.matrix[x2][y2] == 7:#hedor y brisa
            self.matrix[x2][y2] = 13
        elif self.matrix[x2][y2] == 6:#hueco
            self.matrix[x2][y2] = 14
        elif self.matrix[x2][y2] == 2:#wumpus
            self.matrix[x2][y2] = 15
        elif self.matrix[x2][y2] == 4:#oro
            self.matrix[x2][y2] = 16
        elif self.matrix[x2][y2] == 8:#hedor y oro
            self.matrix[x2][y2] = 17
        elif self.matrix[x2][y2] == 9:#hedor brisa y oro
            self.matrix[x2][y2] = 18
        elif self.matrix[x2][y2] == 10:#brisa y oro
            self.matrix[x2][y2] = 19

    def lanzarFlecha(self, eje):#0 -x / 1 +x / 2 -y / 3 +y
        self.flechas -= 1
        for x, fila in enumerate(self.matrix):
            for y, casilla in enumerate(fila):
                if casilla == 2:
                    posWumpus = (x, y)
                elif casilla in [1, 11, 12, 13, 14, 15, 16, 17, 18, 19]:
                    posAgente = (x, y)
        #Miramos en el eje x             
        if posWumpus[0] == posAgente[0]:
            if posWumpus[1] - posAgente[1] < 0:
                if eje == 2:
                    self.matrix[posWumpus[0]][posWumpus[1]] = 0 #Ha matado al wumpus
                    self.eliminarHedor(*posWumpus)
            else:
                if eje == 3:
                    self.matrix[posWumpus[0]][posWumpus[1]] = 0 #Ha matado al wumpus
                    self.eliminarHedor(*posWumpus)
                    
        #Miramos en el eje y
        if posWumpus[1] == posAgente[1]:
            if posWumpus[0] - posAgente[0] < 0:
                if eje == 0:
                    self.matrix[posWumpus[0]][posWumpus[1]] = 0 #Ha matado al wumpus
                    self.eliminarHedor(*posWumpus)
            else:
                if eje == 1:
                    self.matrix[posWumpus[0]][posWumpus[1]] = 0 #Ha matado al wumpus
                    self.eliminarHedor(*posWumpus)

    def eliminarHedor(self, x, y):
        
        casillasHedor = self.posiciones_adyacentes(x, y, len(self.matrix))

        for i, j in casillasHedor:

            if self.matrix[i][j] == 3:#Solo hedor
                self.matrix[i][j] = 0
            elif self.matrix[i][j] == 7:#Hedor + brisa
                self.matrix[i][j] = 5
            elif self.matrix[i][j] == 8:#Hedor + oro
                self.matrix[i][j] = 4
            elif self.matrix[i][j] == 9:#Hedor + brisa + oro
                self.matrix[i][j] = 10
            elif self.matrix[i][j] == 11:#Agente + hedor
                self.matrix[i][j] = 1
            elif self.matrix[i][j] == 13:#Agente + hedor + brisa
                self.matrix[i][j] = 12
            elif self.matrix[i][j] == 17:#Agente + hedor + oro
                self.matrix[i][j] = 16
            elif self.matrix[i][j] == 18:#Agente + hedor + brisa + oro
                self.matrix[i][j] = 19

    def encontrarWumpus(self):
        for fila in self.matrix:
            for casilla in fila:
                if casilla == 2:
                    return True
        return False

In [3]:
class TableroGrafico:

    def __init__(self, tablero = None):
        self.tablero = tablero

    def cargarFichero(self, dimension):
        
        # Cargamos un fichero que codifica un tablero como el indicado arriba.
        if dimension == 4:
            fileHandler = open ("TableroWumpus4x4.txt", "r",encoding="utf-8") #Para cambiar el tamaño del tablero basta con introducir un ".txt" con diferente tamaño
        else:
            fileHandler = open ("TableroWumpus6x6.txt", "r",encoding="utf-8")
        # Get list of all lines in file
        listOfLines = fileHandler.readlines()
     
        # Close file 
        fileHandler.close()

        return listOfLines

    def getTablero(self, dimension):
        if self.tablero is None:
            self.construirTablero(dimension)
        return self.tablero

    def getCodigoTablero(self, caracter):
        '''
        Devuelve el código de mapa asociado a los caracteres definidos.
        '''    
        
        codigo = 0
        if caracter == 'A':
            codigo = 1
        elif caracter == 'W':
            codigo = 2        
        elif caracter == 'H':
            codigo = 3
        elif caracter == 'O':
            codigo = 4
        elif caracter == 'S':
            codigo = 5
        elif caracter == 'C':
            codigo = 6
        elif caracter == 'HS':
            codigo = 7
        elif caracter == 'HO':
            codigo = 8
        elif caracter == 'HSO':
            codigo = 9
        elif caracter == 'SO':
            codigo = 10
        elif caracter == 'AH':
            codigo = 11
        elif caracter == 'AS':
            codigo = 12
        elif caracter == 'AHS':
            codigo = 13
        elif caracter == 'AC':
            codigo = 14
        elif caracter == 'AW':
            codigo = 15
        elif caracter == 'AO':
            codigo = 16
        elif caracter == 'AHO':
            codigo = 17
        elif caracter == 'AHSO':
            codigo = 18
        elif caracter == 'ASO':
            codigo = 19
            
        return codigo

    def construirTablero(self, dimension):
        self.tablero = []
        nFila=0
        listOfLines = self.cargarFichero(dimension)
        ancho=len(listOfLines) 
        
        for linea in listOfLines:
                
                fila = list(itertools.repeat(0, ancho))
        
                nCol = 0
                for ch in linea:
                    if ch =='\n':
                        break
        
                    codigo = self.getCodigoTablero(ch)
                    fila[nCol] = codigo                
        
                    nCol+=1
        
                self.tablero.append(fila)
                nFila+=1

    def get_content(self, coord):
        """
        Obtiene el contenido de una determinada posición.
        
        Parameters
        ----------
        coord : Posición [y,x] de la que queremos conocer el contenido
        
        Returns
        --------
        contenido : Una lista de tamaño 1 .
        """
        contenido = [None]
        
        if self.tablero[coord[0]][coord[1]] == 0:
            contenido[0] = "CasillaVacia"
        elif self.tablero[coord[0]][coord[1]] == 1:
            contenido[0] = "CasillaAgente"
        elif self.tablero[coord[0]][coord[1]] == 2:
            contenido[0] = "CasillaWumpus"
        elif self.tablero[coord[0]][coord[1]] == 3:
            contenido[0] = "CasillaHedor"
        elif self.tablero[coord[0]][coord[1]] == 4:
            contenido[0] = "CasillaOro"
        elif self.tablero[coord[0]][coord[1]] == 5:
            contenido[0] = "CasillaBrisa"
        elif self.tablero[coord[0]][coord[1]] == 6:
            contenido[0] = "CasillaHueco"
        elif self.tablero[coord[0]][coord[1]] == 7:
            contenido[0] = "CasillaHedorBrisa"
        elif self.tablero[coord[0]][coord[1]] == 8:
            contenido[0] = "CasillaHedorOro"
        elif self.tablero[coord[0]][coord[1]] == 9:
            contenido[0] = "CasillaHedorBrisaOro"
        elif self.tablero[coord[0]][coord[1]] == 10:
            contenido[0] = "CasillaBrisaOro"
        elif self.tablero[coord[0]][coord[1]] == 11:
            contenido[0] = "CasillaAgenteHedor"
        elif self.tablero[coord[0]][coord[1]] == 12:
            contenido[0] = "CasillaAgenteBrisa"
        elif self.tablero[coord[0]][coord[1]] == 13:
            contenido[0] = "CasillaAgenteHedorBrisa"
        elif self.tablero[coord[0]][coord[1]] == 14:
            contenido[0] = "CasillaAgenteHueco"
        elif self.tablero[coord[0]][coord[1]] == 15:
            contenido[0] = "CasillaAgenteWumpus"
        elif self.tablero[coord[0]][coord[1]] == 16:
            contenido[0] = "CasillaAgenteOro"
        elif self.tablero[coord[0]][coord[1]] == 17:
            contenido[0] = "CasillaAgenteHedorOro"
        elif self.tablero[coord[0]][coord[1]] == 18:
            contenido[0] = "CasillaAgenteHedorBrisaOro"
        elif self.tablero[coord[0]][coord[1]] == 19:
            contenido[0] = "CasillaAgenteBrisaOro"
        
        return contenido

    

    def get_html(self, dimension):
        """ Muestra una representación gráfica del juego.
    
        Devuelve un "string" que contiene HTML
    
        """ 
        if self.tablero is None:
            self.construirTablero(dimension)
            
        element_image = {
        "CasillaVacia": "./ImagenesCasillasWumpus/CasillaVacia.png",
        "CasillaAgente": "./ImagenesCasillasWumpus/CasillaAgente.png",
        "CasillaWumpus": "./ImagenesCasillasWumpus/CasillaWumpus.png",
        "CasillaHedor": "./ImagenesCasillasWumpus/CasillaHedor.png",
        "CasillaOro": "./ImagenesCasillasWumpus/CasillaOro.png",
        "CasillaBrisa": "./ImagenesCasillasWumpus/CasillaBrisa.png",
        "CasillaHueco": "./ImagenesCasillasWumpus/CasillaHueco.png",
        "CasillaHedorBrisa": "./ImagenesCasillasWumpus/CasillaHedorBrisa.png",
        "CasillaHedorOro": "./ImagenesCasillasWumpus/CasillaHedorOro.png",
        "CasillaHedorBrisaOro": "./ImagenesCasillasWumpus/CasillaHedorBrisaOro.png",
        "CasillaBrisaOro": "./ImagenesCasillasWumpus/CasillaBrisaOro.png",
        "CasillaAgenteHedor": "./ImagenesCasillasWumpus/CasillaAgenteHedor.png",
        "CasillaAgenteBrisa": "./ImagenesCasillasWumpus/CasillaAgenteBrisa.png",
        "CasillaAgenteHedorBrisa": "./ImagenesCasillasWumpus/CasillaAgenteHedorBrisa.png",
        "CasillaAgenteHueco": "./ImagenesCasillasWumpus/CasillaAgenteHueco.png",
        "CasillaAgenteWumpus": "./ImagenesCasillasWumpus/CasillaAgenteWumpus.png",
        "CasillaAgenteOro": "./ImagenesCasillasWumpus/CasillaAgenteOro.png",
        "CasillaAgenteHedorOro": "./ImagenesCasillasWumpus/CasillaAgenteHedorOro.png",
        "CasillaAgenteHedorBrisaOro": "./ImagenesCasillasWumpus/CasillaAgenteHedorBrisaOro.png",
        "CasillaAgenteBrisaOro": "./ImagenesCasillasWumpus/CasillaAgenteBrisaOro.png"
        }
        
        height = len(self.tablero)
        width = len(self.tablero[0])
        
        html_string = "<style> img.game {width: 37px !important; height: 22px !important;}</style><table>"
        
    
    
        new_row = "<tr>"
        end_row = "</tr>"
        
        for i in range(height):
            html_string+=new_row
            for j in range(width):
                
                content = self.get_content((i,j))
                drawing = element_image[content[0]]
                
                html = '<td><img class="game" src=%s alt=""></img></td>' % drawing     
                
                
                html_string+=html
            html_string+=end_row
                
        html_string += "</table>"
            
                    
        return html_string
        
    def mostrarTablero(self, dimension):
        display(HTML(self.get_html(dimension)))

In [4]:
class Partida:


    def generarSucesoresMax(self, state):
        ejes = [0, 1, 2, 3]
        sucesores = []
        movimientosDisponibles = state.movimientosAgenteDisponibles()
        for casilla in movimientosDisponibles:
            new_state = Tablerowumpus(state.getMatrix(), state.getFlechas())
            new_state.moverAgente(casilla)
            sucesores.append(new_state)
            
        if state.getFlechas() > 0:#Si puede lanzar flecha
            for eje in ejes:
                new_state = Tablerowumpus(state.getMatrix(), state.getFlechas())
                new_state.lanzarFlecha(eje)
                sucesores.append(new_state)
            
        return sucesores

    def generarSucesoresMin(self, state):

        sucesores = []

        while len(sucesores) < 20: #Generamos 20 sucesores aleatorios, 20 movimientos de huecos aleatorios
            new_state = Tablerowumpus(state.getMatrix(), state.getFlechas())
            new_state.moverHuecos()
            sucesores.append(new_state)

        return sucesores
            

    def maximize(self, state, alfa, beta, currentLevel, maxLevel):

        if state.gameOver() or currentLevel == maxLevel:
            return (None, state.utility())

        max_state, max_score = None, -math.inf

        sucesores = self.generarSucesoresMax(state)
        for sucesor in sucesores:
            _, score = self.minimize(sucesor, alfa, beta, currentLevel+1, maxLevel)
            if score > max_score:
                max_state, max_score = sucesor, score
            if max_score >= beta:
                break
            if max_score > alfa:
                alfa = max_score

        return max_state, max_score

    def minimize(self, state, alfa, beta, currentLevel, maxLevel):

        if state.gameOver() or currentLevel == maxLevel:
            return (None, state.utility())

        min_state, min_score = None, math.inf

        sucesores = self.generarSucesoresMin(state)
        for sucesor in sucesores:
            _, score = self.maximize(sucesor, alfa, beta, currentLevel+1, maxLevel)
            if score < min_score:
                min_state, min_score = sucesor, score
            if min_score <= alfa:
                break
            if min_score < beta:
                beta = min_score

        return min_state, min_score

    def minimax(self, state):
        max_state, _ = self.maximize(state, -math.inf, math.inf, 0, 3)
        return max_state


    

In [5]:


# Imprimir "COMIENZO PARTIDA" en un estilo grande
print("#########################################")
print("#                                       #")
print("#           COMIENZO PARTIDA            #")
print("#                                       #")
print("#########################################")
print()
    
# Preguntar al usuario por el tamaño del tablero
while True:
    print("Elige el tamaño del tablero:")
    print("1. 4x4")
    print("2. 6x6")
    opcion = input("Selecciona 1 o 2: ")
        
    if opcion == "1":
        print()
        print("Has seleccionado un tablero de 4x4.")
        print()
        dimension = 4
        break
    elif opcion == "2":
        print()
        print("Has seleccionado un tablero de 6x6.")
        print()
        dimension = 6
        break
    else:
        print("Opción no válida. Por favor, selecciona 1 o 2.")
        

while True:
    print()
    print("Elige una opción para el tablero inicial:")
    print()
    print("1. Generar tablero inicial aleatoriamente (recomendada).")
    print("2. Introducir tablero por pantalla (el usuario se tiene que encargar de crear un tablero correcto).")
    print("3. Utilizar un tablero prefabricado guardado como .txt.")
    print()
    opcion = input("Selecciona 1, 2 o 3: ")
        
    if opcion == "1":
        state = Tablerowumpus()
        state.generarTableroInicial(dimension)
        
        break
    elif opcion == "2":
        print()
        print("A continuacion, introduce las casillas del tablero por teclado. Abajo le aparecera el significado de cada codigo.")
        print()
        print("0 = CasillaVacia")
        print("1 = CasillaAgente")
        print("2 = CasillaWumpus")
        print("3 = CasillaHedor")
        print("4 = CasillaOro")
        print("5 = CasillaBrisa")
        print("6 = CasillaHueco")
        print("7 = CasillaHedorBrisa")
        print("8 = CasillaHedorOro")
        print("9 = CasillaHedorBrisaOro")
        print("10 = CasillaBrisaOro")
        print("11 = CasillaAgenteHedor")
        print("12 = CasillaAgenteBrisa")
        print("13 = CasillaAgenteHedorBrisa")
        print("14 = CasillaAgenteHueco")
        print("15 = CasillaAgenteWumpus")
        print("16 = CasillaAgenteOro")
        print("17 = CasillaAgenteHedorOro")
        print("18 = CasillaAgenteHedorBrisaOro")
        print("19 = CasillaAgenteBrisaOro")

        print("Introduzca las casillas:")
        tablero = [[0 for _ in range(dimension)] for _ in range(dimension)]
        for i in range(dimension):
            for j in range(dimension):
                tablero[i][j] = int(input(f"Introduce el contenido de la casilla ({i}, {j}): "))
        print()
        print("Contenido introducido correctamente.")
        state = Tablerowumpus(tablero) 
        break
    elif opcion == "3":
        tableroGrafico = TableroGrafico()
        state = Tablerowumpus(tableroGrafico.getTablero(dimension))
        print()
        print("Se ha seleccionado un tablero prefabricado de dimension ", dimension,"x",dimension," correctamente.")
        print()
        break
    else:
        print("Opción no válida. Por favor, selecciona 1, 2 o 3.")


print()
print("        TABLERO INICIAL         ")
print()

partida = Partida()
tableroGrafico = TableroGrafico(state.getMatrix())
tableroGrafico.mostrarTablero(dimension)
flag = True
turno = 0
flechaDisponible = True

while not state.gameOver() and flag:

    if turno == 0 and not state.gameOver():
        
        
        # Preguntar al usuario si desea continuar
        while True:
            respuesta = input("¿Deseas continuar? (s/n): ").strip().lower()
            if respuesta == 's':
                print("#########################################")
                print("#                                       #")
                print("#           TURNO DEL AGENTE            #")
                print("#                                       #")
                print("#########################################")
                print()
                
                state = partida.minimax(state)
                tableroGrafico = TableroGrafico(state.getMatrix())
                tableroGrafico.mostrarTablero(dimension)
                
                if flechaDisponible:
                    if state.getFlechas() < 1:
                        flechaDisponible = False
                        print("El Agente ha lanzado la flecha yyy.....")
                        if state.encontrarWumpus():
                            print("HA FALLADO EL TIRO! Tendra que afinar esa punteria...")
                        else:
                            print("HA ACERTADO!!! AL DIABLO CON ESE TAL WUMPUS!")
                        
                turno = 1
                break
            elif respuesta == 'n':
                print("Partida terminada. Saliendo...")
                flag = False
                break
            else:
                print("Opción no válida. Por favor, responde con 's' o 'n'.")

    if turno == 1 and not state.gameOver():
        
        # Preguntar al usuario si desea continuar
        while True:
            respuesta = input("¿Deseas continuar? (s/n): ").strip().lower()
            if respuesta == 's':
                print("#########################################")
                print("#                                       #")
                print("#         TURNO DE LOS HUECOS           #")
                print("#                                       #")
                print("#########################################")
                print()

                state.moverHuecos()
                tableroGrafico = TableroGrafico(state.getMatrix())
                tableroGrafico.mostrarTablero(dimension)
                
                turno = 0
                break
            elif respuesta == 'n':
                print("Partida terminada. Saliendo...")
                flag = False
                break
            else:
                print("Opción no válida. Por favor, responde con 's' o 'n'.")


for fila in state.getMatrix():
    for casilla in fila:
        if casilla == 15:
            print()
            print("El Wumpus ha matado al agente...")
            print()
        if casilla in [16, 17, 18, 19]:
            print()
            print("El agente ha robado el oro!!!")
            print()
            
print("EL JUEGO HA TERMINADO")

#########################################
#                                       #
#           COMIENZO PARTIDA            #
#                                       #
#########################################

Elige el tamaño del tablero:
1. 4x4
2. 6x6


Selecciona 1 o 2:  1



Has seleccionado un tablero de 4x4.


Elige una opción para el tablero inicial:

1. Generar tablero inicial aleatoriamente (recomendada).
2. Introducir tablero por pantalla (el usuario se tiene que encargar de crear un tablero correcto).
3. Utilizar un tablero prefabricado guardado como .txt.



Selecciona 1, 2 o 3:  3



Se ha seleccionado un tablero prefabricado de dimension  4 x 4  correctamente.


        TABLERO INICIAL         



...

,,,
,,,
,,,
,,,


¿Deseas continuar? (s/n):  s


#########################################
#                                       #
#           TURNO DEL AGENTE            #
#                                       #
#########################################



,,,
,,,
,,,
,,,



El agente ha robado el oro!!!

EL JUEGO HA TERMINADO
